In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from scipy.optimize import curve_fit
import shap
from tqdm import tqdm
import time, gc

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

F:\ProgramData\miniconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
col_labels = ['label_close2max',
'label_close3max',
'label_close5max',
'label_close10max',
'label_close15max',
'label_close20max',
'label_close2min',
'label_close3min',
'label_close5min',
'label_close10min',
'label_close15min',
'label_close20min',
'label_close2mean',
'label_close3mean',
'label_close5mean',
'label_close10mean',
'label_close15mean',
'label_close20mean',
'inc_close2mean_perc5',
'inc_close2mean_perc10',
'inc_close2mean_perc3',
'inc_close2max_perc5',
'inc_close2max_perc10',
'inc_close2max_perc3',
'inc_close3mean_perc5',
'inc_close3mean_perc10',
'inc_close3mean_perc3',
'inc_close3max_perc5',
'inc_close3max_perc10',
'inc_close3max_perc3',
'inc_close5mean_perc5',
'inc_close5mean_perc10',
'inc_close5mean_perc3',
'inc_close5max_perc5',
'inc_close5max_perc10',
'inc_close5max_perc3',
'inc_close10mean_perc5',
'inc_close10mean_perc10',
'inc_close10mean_perc3',
'inc_close10max_perc5',
'inc_close10max_perc10',
'inc_close10max_perc3',
'dec_close2mean_perc5',
'dec_close2mean_perc10',
'dec_close2mean_perc3',
'dec_close2min_perc5',
'dec_close2min_perc10',
'dec_close2min_perc3',
'dec_close3mean_perc5',
'dec_close3mean_perc10',
'dec_close3mean_perc3',
'dec_close3min_perc5',
'dec_close3min_perc10',
'dec_close3min_perc3',
'dec_close5mean_perc5',
'dec_close5mean_perc10',
'dec_close5mean_perc3',
'dec_close5min_perc5',
'dec_close5min_perc10',
'dec_close5min_perc3',
'dec_close10mean_perc5',
'dec_close10mean_perc10',
'dec_close10mean_perc3',
'dec_close10min_perc5',
'dec_close10min_perc10',
'dec_close10min_perc3',
'inc_close2mean_perc7',
'inc_close2max_perc7',
'inc_close3mean_perc7',
'inc_close3max_perc7',
'inc_close5mean_perc7',
'inc_close5max_perc7',
'inc_close10mean_perc7',
'inc_close10max_perc7',
'dec_close2mean_perc7',
'dec_close2min_perc7',
'dec_close3mean_perc7',
'dec_close3min_perc7',
'dec_close5mean_perc7',
'dec_close5min_perc7',
'dec_close10mean_perc7',
'dec_close10min_perc7']

In [3]:
data = pd.read_parquet("datas//ml_data//ml_feature_label_BAC.parquet")

In [28]:
data_used = data[['open', 'close', 'high', 'low', 'volume']].copy()

In [39]:
data_used.ta.sma(append=True)

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
6282    41.557
6283    41.794
6284    42.009
6285    42.277
6286    42.271
Name: SMA_10, Length: 6287, dtype: float64

In [40]:
data_used.head(5)

,open,close,high,low,volume,STOCHk_14_3_3,STOCHd_14_3_3,ISA_9,ISB_26,ITS_9,IKS_26,RVI_14,AROOND_3,AROONU_3,AROONOSC_3,PPO_12_26_9,PPOh_12_26_9,PPOs_12_26_9,SMA_10
0,64.50,64.87,65.19,63.94,4018200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64.94,64.25,65.75,64.19,3909800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64.75,63.00,64.75,62.94,4876700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63.88,63.50,64.50,63.50,5136800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.666667,33.333333,-33.333333,NaN,NaN,NaN,NaN
4,65.50,65.06,66.87,65.00,4981800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.333333,100.000000,66.666667,NaN,NaN,NaN,NaN


In [22]:
for i in data.columns:
    print(f"'{i}',")

'date',
'close',
'open',
'high',
'low',
'volume',
'RSI_3',
'RSI_5',
'RSI_10',
'RSI_15',
'RSI_20',
'RSI_60',
'RSI_120',
'EMA_3',
'EMA_5',
'EMA_10',
'EMA_15',
'EMA_20',
'EMA_60',
'EMA_120',
'SMA_3',
'SMA_5',
'SMA_10',
'SMA_15',
'SMA_20',
'SMA_60',
'SMA_120',
'BBL_3_2.0',
'BBM_3_2.0',
'BBU_3_2.0',
'BBB_3_2.0',
'BBP_3_2.0',
'BBL_5_2.0',
'BBM_5_2.0',
'BBU_5_2.0',
'BBB_5_2.0',
'BBP_5_2.0',
'BBL_10_2.0',
'BBM_10_2.0',
'BBU_10_2.0',
'BBB_10_2.0',
'BBP_10_2.0',
'BBL_15_2.0',
'BBM_15_2.0',
'BBU_15_2.0',
'BBB_15_2.0',
'BBP_15_2.0',
'BBL_20_2.0',
'BBM_20_2.0',
'BBU_20_2.0',
'BBB_20_2.0',
'BBP_20_2.0',
'BBL_60_2.0',
'BBM_60_2.0',
'BBU_60_2.0',
'BBB_60_2.0',
'BBP_60_2.0',
'BBL_120_2.0',
'BBM_120_2.0',
'BBU_120_2.0',
'BBB_120_2.0',
'BBP_120_2.0',
'ADX_3',
'DMP_3',
'DMN_3',
'ADX_5',
'DMP_5',
'DMN_5',
'ADX_10',
'DMP_10',
'DMN_10',
'ADX_15',
'DMP_15',
'DMN_15',
'ADX_20',
'DMP_20',
'DMN_20',
'ADX_60',
'DMP_60',
'DMN_60',
'ADX_120',
'DMP_120',
'DMN_120',
'OBV',
'STOCHk_14_3_3',
'STOCHd_14_3_3',
'ISA_9',
'

In [5]:
col_kept = [icol for icol in data.columns if icol not in col_labels]

In [6]:
data = data.dropna(subset=col_kept)

In [7]:
data['day'] = data['date'].dt.strftime("%Y%m%d").astype('int')

In [8]:
data.head()

,date,close,open,high,low,volume,RSI_3,RSI_5,RSI_10,RSI_15,RSI_20,RSI_60,RSI_120,EMA_3,EMA_5,EMA_10,EMA_15,EMA_20,EMA_60,EMA_120,SMA_3,SMA_5,SMA_10,SMA_15,SMA_20,SMA_60,SMA_120,BBL_3_2.0,BBM_3_2.0,BBU_3_2.0,BBB_3_2.0,BBP_3_2.0,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,BBL_10_2.0,BBM_10_2.0,BBU_10_2.0,BBB_10_2.0,BBP_10_2.0,BBL_15_2.0,BBM_15_2.0,BBU_15_2.0,BBB_15_2.0,BBP_15_2.0,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,BBL_60_2.0,BBM_60_2.0,BBU_60_2.0,BBB_60_2.0,BBP_60_2.0,BBL_120_2.0,BBM_120_2.0,BBU_120_2.0,BBB_120_2.0,BBP_120_2.0,ADX_3,DMP_3,DMN_3,ADX_5,DMP_5,DMN_5,ADX_10,DMP_10,DMN_10,ADX_15,DMP_15,DMN_15,ADX_20,DMP_20,DMN_20,ADX_60,DMP_60,DMN_60,ADX_120,DMP_120,DMN_120,OBV,STOCHk_14_3_3,STOCHd_14_3_3,ISA_9,ISB_26,ITS_9,IKS_26,MACD_5_10_9,MACDh_5_10_9,MACDs_5_10_9,grad_MACD_5_10_9,diff_MACD_5_10_9,grad_MACDh_5_10_9,diff_MACDh_5_10_9,grad_MACDs_5_10_9,diff_MACDs_5_10_9,grad_DMP_3,grad_DMN_3,grad_RSI_3,diff_RSI_3,grad_EMA_3,diff_EMA_3,grad_BBL_3,diff_BBL_3,grad_BBM_3,diff_BBM_3,grad_BBU_3,diff_BBU_3,grad_BBB_3,diff_BBB_3,grad_BBP_3,diff_BBP_3,r_close_bbl_3,r_close_bbu_3,r_close_bbm_3,r_close_bbb_3,r_close_bbp_3,r_open_bbl_3,r_open_bbu_3,r_open_bbm_3,r_open_bbb_3,r_open_bbp_3,r_low_bbl_3,r_low_bbu_3,r_low_bbm_3,r_low_bbb_3,r_low_bbp_3,r_high_bbl_3,r_high_bbu_3,r_high_bbm_3,r_high_bbb_3,r_high_bbp_3,r_close_rsi_3,r_open_rsi_3,r_low_rsi_3,r_high_rsi_3,r_close_EMA_3,r_open_EMA_3,r_low_EMA_3,r_high_EMA_3,r_close_SMA_3,r_open_SMA_3,r_low_SMA_3,r_high_SMA_3,r_sma3_bbl_3,r_sma3_bbu_3,r_sma3_bbm_3,r_sma3_bbb_3,r_sma3_bbp_3,r_sma5_bbl_3,r_sma5_bbu_3,r_sma5_bbm_3,r_sma5_bbb_3,r_sma5_bbp_3,r_dmp_adx_3,r_dmn_adx_3,grad_DMP_5,grad_DMN_5,grad_RSI_5,diff_RSI_5,grad_EMA_5,diff_EMA_5,grad_BBL_5,diff_BBL_5,grad_BBM_5,diff_BBM_5,grad_BBU_5,diff_BBU_5,grad_BBB_5,diff_BBB_5,grad_BBP_5,diff_BBP_5,r_close_bbl_5,r_close_bbu_5,r_close_bbm_5,r_close_bbb_5,r_close_bbp_5,r_open_bbl_5,r_open_bbu_5,r_open_bbm_5,r_open_bbb_5,r_open_bbp_5,r_low_bbl_5,r_low_bbu_5,r_low_bbm_5,r_low_bbb_5,r_low_bbp_5,r_high_bbl_5,r_high_bbu_5,r_high_bbm_5,r_high_bbb_5,r_high_bbp_5,r_close_rsi_5,r_open_rsi_5,r_low_rsi_5,r_high_rsi_5,r_close_EMA_5,r_open_EMA_5,r_low_EMA_5,r_high_EMA_5,r_close_SMA_5,r_open_SMA_5,r_low_SMA_5,r_high_SMA_5,r_sma3_bbl_5,r_sma3_bbu_5,r_sma3_bbm_5,r_sma3_bbb_5,r_sma3_bbp_5,r_sma5_bbl_5,r_sma5_bbu_5,r_sma5_bbm_5,r_sma5_bbb_5,r_sma5_bbp_5,r_dmp_adx_5,r_dmn_adx_5,grad_DMP_10,grad_DMN_10,grad_RSI_10,diff_RSI_10,grad_EMA_10,diff_EMA_10,grad_BBL_10,diff_BBL_10,grad_BBM_10,diff_BBM_10,grad_BBU_10,diff_BBU_10,grad_BBB_10,diff_BBB_10,grad_BBP_10,diff_BBP_10,r_close_bbl_10,r_close_bbu_10,r_close_bbm_10,r_close_bbb_10,r_close_bbp_10,r_open_bbl_10,r_open_bbu_10,r_open_bbm_10,r_open_bbb_10,r_open_bbp_10,r_low_bbl_10,r_low_bbu_10,r_low_bbm_10,r_low_bbb_10,r_low_bbp_10,r_high_bbl_10,r_high_bbu_10,r_high_bbm_10,r_high_bbb_10,r_high_bbp_10,r_close_rsi_10,r_open_rsi_10,r_low_rsi_10,r_high_rsi_10,r_close_EMA_10,r_open_EMA_10,r_low_EMA_10,r_high_EMA_10,r_close_SMA_10,r_open_SMA_10,r_low_SMA_10,r_high_SMA_10,r_sma3_bbl_10,r_sma3_bbu_10,r_sma3_bbm_10,r_sma3_bbb_10,r_sma3_bbp_10,r_sma5_bbl_10,r_sma5_bbu_10,r_sma5_bbm_10,r_sma5_bbb_10,r_sma5_bbp_10,r_dmp_adx_10,r_dmn_adx_10,grad_DMP_15,grad_DMN_15,grad_RSI_15,diff_RSI_15,grad_EMA_15,diff_EMA_15,grad_BBL_15,diff_BBL_15,grad_BBM_15,diff_BBM_15,grad_BBU_15,diff_BBU_15,grad_BBB_15,diff_BBB_15,grad_BBP_15,diff_BBP_15,r_close_bbl_15,r_close_bbu_15,r_close_bbm_15,r_close_bbb_15,r_close_bbp_15,r_open_bbl_15,r_open_bbu_15,r_open_bbm_15,r_open_bbb_15,r_open_bbp_15,r_low_bbl_15,r_low_bbu_15,r_low_bbm_15,r_low_bbb_15,r_low_bbp_15,r_high_bbl_15,r_high_bbu_15,r_high_bbm_15,r_high_bbb_15,r_high_bbp_15,r_close_rsi_15,r_open_rsi_15,r_low_rsi_15,r_high_rsi_15,r_close_EMA_15,r_open_EMA_15,r_low_EMA_15,r_high_EMA_15,r_close_SMA_15,r_open_SMA_15,r_low_SMA_15,r_high_SMA_15,r_sma3_bbl_15,r_sma3_bbu_15,r_sma3_bbm_15,r_sma3_bbb_15,r_sma3_bbp_15,r_sma5_bbl_15,r_sma5_bbu_15,r_sma5_bbm_15,r_sma5_bbb_15,r_sma5_bbp_15,r_dmp_adx_15,r_dmn_

# Metadata

In [9]:
cols_feat = ['RSI_3',
'RSI_5',
'RSI_10',
'RSI_15',
'RSI_20',
'RSI_60',
'RSI_120',
'ADX_3',
'DMP_3',
'DMN_3',
'ADX_5',
'DMP_5',
'DMN_5',
'ADX_10',
'DMP_10',
'DMN_10',
'ADX_15',
'DMP_15',
'DMN_15',
'ADX_20',
'DMP_20',
'DMN_20',
'ADX_60',
'DMP_60',
'DMN_60',
'ADX_120',
'DMP_120',
'DMN_120',
'OBV',
'STOCHk_14_3_3',
'STOCHd_14_3_3',
'ISA_9',
'ISB_26',
'ITS_9',
'IKS_26',
'grad_MACD_5_10_9',
'diff_MACD_5_10_9',
'grad_MACDh_5_10_9',
'diff_MACDh_5_10_9',
'grad_MACDs_5_10_9',
'diff_MACDs_5_10_9',
'grad_DMP_3',
'grad_DMN_3',
'grad_RSI_3',
'diff_RSI_3',
'grad_EMA_3',
'diff_EMA_3',
'grad_BBL_3',
'diff_BBL_3',
'grad_BBM_3',
'diff_BBM_3',
'grad_BBU_3',
'diff_BBU_3',
'grad_BBB_3',
'diff_BBB_3',
'grad_BBP_3',
'diff_BBP_3',
'r_close_bbl_3',
'r_close_bbu_3',
'r_close_bbm_3',
'r_close_bbb_3',
'r_close_bbp_3',
'r_open_bbl_3',
'r_open_bbu_3',
'r_open_bbm_3',
'r_open_bbb_3',
'r_open_bbp_3',
'r_low_bbl_3',
'r_low_bbu_3',
'r_low_bbm_3',
'r_low_bbb_3',
'r_low_bbp_3',
'r_high_bbl_3',
'r_high_bbu_3',
'r_high_bbm_3',
'r_high_bbb_3',
'r_high_bbp_3',
'r_close_rsi_3',
'r_open_rsi_3',
'r_low_rsi_3',
'r_high_rsi_3',
'r_close_EMA_3',
'r_open_EMA_3',
'r_low_EMA_3',
'r_high_EMA_3',
'r_close_SMA_3',
'r_open_SMA_3',
'r_low_SMA_3',
'r_high_SMA_3',
'r_sma3_bbl_3',
'r_sma3_bbu_3',
'r_sma3_bbm_3',
'r_sma3_bbb_3',
'r_sma3_bbp_3',
'r_sma5_bbl_3',
'r_sma5_bbu_3',
'r_sma5_bbm_3',
'r_sma5_bbb_3',
'r_sma5_bbp_3',
'r_dmp_adx_3',
'r_dmn_adx_3',
'grad_DMP_5',
'grad_DMN_5',
'grad_RSI_5',
'diff_RSI_5',
'grad_EMA_5',
'diff_EMA_5',
'grad_BBL_5',
'diff_BBL_5',
'grad_BBM_5',
'diff_BBM_5',
'grad_BBU_5',
'diff_BBU_5',
'grad_BBB_5',
'diff_BBB_5',
'grad_BBP_5',
'diff_BBP_5',
'r_close_bbl_5',
'r_close_bbu_5',
'r_close_bbm_5',
'r_close_bbb_5',
'r_close_bbp_5',
'r_open_bbl_5',
'r_open_bbu_5',
'r_open_bbm_5',
'r_open_bbb_5',
'r_open_bbp_5',
'r_low_bbl_5',
'r_low_bbu_5',
'r_low_bbm_5',
'r_low_bbb_5',
'r_low_bbp_5',
'r_high_bbl_5',
'r_high_bbu_5',
'r_high_bbm_5',
'r_high_bbb_5',
'r_high_bbp_5',
'r_close_rsi_5',
'r_open_rsi_5',
'r_low_rsi_5',
'r_high_rsi_5',
'r_close_EMA_5',
'r_open_EMA_5',
'r_low_EMA_5',
'r_high_EMA_5',
'r_close_SMA_5',
'r_open_SMA_5',
'r_low_SMA_5',
'r_high_SMA_5',
'r_sma3_bbl_5',
'r_sma3_bbu_5',
'r_sma3_bbm_5',
'r_sma3_bbb_5',
'r_sma3_bbp_5',
'r_sma5_bbl_5',
'r_sma5_bbu_5',
'r_sma5_bbm_5',
'r_sma5_bbb_5',
'r_sma5_bbp_5',
'r_dmp_adx_5',
'r_dmn_adx_5',
'grad_DMP_10',
'grad_DMN_10',
'grad_RSI_10',
'diff_RSI_10',
'grad_EMA_10',
'diff_EMA_10',
'grad_BBL_10',
'diff_BBL_10',
'grad_BBM_10',
'diff_BBM_10',
'grad_BBU_10',
'diff_BBU_10',
'grad_BBB_10',
'diff_BBB_10',
'grad_BBP_10',
'diff_BBP_10',
'r_close_bbl_10',
'r_close_bbu_10',
'r_close_bbm_10',
'r_close_bbb_10',
'r_close_bbp_10',
'r_open_bbl_10',
'r_open_bbu_10',
'r_open_bbm_10',
'r_open_bbb_10',
'r_open_bbp_10',
'r_low_bbl_10',
'r_low_bbu_10',
'r_low_bbm_10',
'r_low_bbb_10',
'r_low_bbp_10',
'r_high_bbl_10',
'r_high_bbu_10',
'r_high_bbm_10',
'r_high_bbb_10',
'r_high_bbp_10',
'r_close_rsi_10',
'r_open_rsi_10',
'r_low_rsi_10',
'r_high_rsi_10',
'r_close_EMA_10',
'r_open_EMA_10',
'r_low_EMA_10',
'r_high_EMA_10',
'r_close_SMA_10',
'r_open_SMA_10',
'r_low_SMA_10',
'r_high_SMA_10',
'r_sma3_bbl_10',
'r_sma3_bbu_10',
'r_sma3_bbm_10',
'r_sma3_bbb_10',
'r_sma3_bbp_10',
'r_sma5_bbl_10',
'r_sma5_bbu_10',
'r_sma5_bbm_10',
'r_sma5_bbb_10',
'r_sma5_bbp_10',
'r_dmp_adx_10',
'r_dmn_adx_10',
'grad_DMP_15',
'grad_DMN_15',
'grad_RSI_15',
'diff_RSI_15',
'grad_EMA_15',
'diff_EMA_15',
'grad_BBL_15',
'diff_BBL_15',
'grad_BBM_15',
'diff_BBM_15',
'grad_BBU_15',
'diff_BBU_15',
'grad_BBB_15',
'diff_BBB_15',
'grad_BBP_15',
'diff_BBP_15',
'r_close_bbl_15',
'r_close_bbu_15',
'r_close_bbm_15',
'r_close_bbb_15',
'r_close_bbp_15',
'r_open_bbl_15',
'r_open_bbu_15',
'r_open_bbm_15',
'r_open_bbb_15',
'r_open_bbp_15',
'r_low_bbl_15',
'r_low_bbu_15',
'r_low_bbm_15',
'r_low_bbb_15',
'r_low_bbp_15',
'r_high_bbl_15',
'r_high_bbu_15',
'r_high_bbm_15',
'r_high_bbb_15',
'r_high_bbp_15',
'r_close_rsi_15',
'r_open_rsi_15',
'r_low_rsi_15',
'r_high_rsi_15',
'r_close_EMA_15',
'r_open_EMA_15',
'r_low_EMA_15',
'r_high_EMA_15',
'r_close_SMA_15',
'r_open_SMA_15',
'r_low_SMA_15',
'r_high_SMA_15',
'r_sma3_bbl_15',
'r_sma3_bbu_15',
'r_sma3_bbm_15',
'r_sma3_bbb_15',
'r_sma3_bbp_15',
'r_sma5_bbl_15',
'r_sma5_bbu_15',
'r_sma5_bbm_15',
'r_sma5_bbb_15',
'r_sma5_bbp_15',
'r_dmp_adx_15',
'r_dmn_adx_15',
'grad_DMP_20',
'grad_DMN_20',
'grad_RSI_20',
'diff_RSI_20',
'grad_EMA_20',
'diff_EMA_20',
'grad_BBL_20',
'diff_BBL_20',
'grad_BBM_20',
'diff_BBM_20',
'grad_BBU_20',
'diff_BBU_20',
'grad_BBB_20',
'diff_BBB_20',
'grad_BBP_20',
'diff_BBP_20',
'r_close_bbl_20',
'r_close_bbu_20',
'r_close_bbm_20',
'r_close_bbb_20',
'r_close_bbp_20',
'r_open_bbl_20',
'r_open_bbu_20',
'r_open_bbm_20',
'r_open_bbb_20',
'r_open_bbp_20',
'r_low_bbl_20',
'r_low_bbu_20',
'r_low_bbm_20',
'r_low_bbb_20',
'r_low_bbp_20',
'r_high_bbl_20',
'r_high_bbu_20',
'r_high_bbm_20',
'r_high_bbb_20',
'r_high_bbp_20',
'r_close_rsi_20',
'r_open_rsi_20',
'r_low_rsi_20',
'r_high_rsi_20',
'r_close_EMA_20',
'r_open_EMA_20',
'r_low_EMA_20',
'r_high_EMA_20',
'r_close_SMA_20',
'r_open_SMA_20',
'r_low_SMA_20',
'r_high_SMA_20',
'r_sma3_bbl_20',
'r_sma3_bbu_20',
'r_sma3_bbm_20',
'r_sma3_bbb_20',
'r_sma3_bbp_20',
'r_sma5_bbl_20',
'r_sma5_bbu_20',
'r_sma5_bbm_20',
'r_sma5_bbb_20',
'r_sma5_bbp_20',
'r_dmp_adx_20',
'r_dmn_adx_20',
'grad_DMP_60',
'grad_DMN_60',
'grad_RSI_60',
'diff_RSI_60',
'grad_EMA_60',
'diff_EMA_60',
'grad_BBL_60',
'diff_BBL_60',
'grad_BBM_60',
'diff_BBM_60',
'grad_BBU_60',
'diff_BBU_60',
'grad_BBB_60',
'diff_BBB_60',
'grad_BBP_60',
'diff_BBP_60',
'r_close_bbl_60',
'r_close_bbu_60',
'r_close_bbm_60',
'r_close_bbb_60',
'r_close_bbp_60',
'r_open_bbl_60',
'r_open_bbu_60',
'r_open_bbm_60',
'r_open_bbb_60',
'r_open_bbp_60',
'r_low_bbl_60',
'r_low_bbu_60',
'r_low_bbm_60',
'r_low_bbb_60',
'r_low_bbp_60',
'r_high_bbl_60',
'r_high_bbu_60',
'r_high_bbm_60',
'r_high_bbb_60',
'r_high_bbp_60',
'r_close_rsi_60',
'r_open_rsi_60',
'r_low_rsi_60',
'r_high_rsi_60',
'r_close_EMA_60',
'r_open_EMA_60',
'r_low_EMA_60',
'r_high_EMA_60',
'r_close_SMA_60',
'r_open_SMA_60',
'r_low_SMA_60',
'r_high_SMA_60',
'r_sma3_bbl_60',
'r_sma3_bbu_60',
'r_sma3_bbm_60',
'r_sma3_bbb_60',
'r_sma3_bbp_60',
'r_sma5_bbl_60',
'r_sma5_bbu_60',
'r_sma5_bbm_60',
'r_sma5_bbb_60',
'r_sma5_bbp_60',
'r_dmp_adx_60',
'r_dmn_adx_60',
'grad_DMP_120',
'grad_DMN_120',
'grad_RSI_120',
'diff_RSI_120',
'grad_EMA_120',
'diff_EMA_120',
'grad_BBL_120',
'diff_BBL_120',
'grad_BBM_120',
'diff_BBM_120',
'grad_BBU_120',
'diff_BBU_120',
'grad_BBB_120',
'diff_BBB_120',
'grad_BBP_120',
'diff_BBP_120',
'r_close_bbl_120',
'r_close_bbu_120',
'r_close_bbm_120',
'r_close_bbb_120',
'r_close_bbp_120',
'r_open_bbl_120',
'r_open_bbu_120',
'r_open_bbm_120',
'r_open_bbb_120',
'r_open_bbp_120',
'r_low_bbl_120',
'r_low_bbu_120',
'r_low_bbm_120',
'r_low_bbb_120',
'r_low_bbp_120',
'r_high_bbl_120',
'r_high_bbu_120',
'r_high_bbm_120',
'r_high_bbb_120',
'r_high_bbp_120',
'r_close_rsi_120',
'r_open_rsi_120',
'r_low_rsi_120',
'r_high_rsi_120',
'r_close_EMA_120',
'r_open_EMA_120',
'r_low_EMA_120',
'r_high_EMA_120',
'r_close_SMA_120',
'r_open_SMA_120',
'r_low_SMA_120',
'r_high_SMA_120',
'r_sma3_bbl_120',
'r_sma3_bbu_120',
'r_sma3_bbm_120',
'r_sma3_bbb_120',
'r_sma3_bbp_120',
'r_sma5_bbl_120',
'r_sma5_bbu_120',
'r_sma5_bbm_120',
'r_sma5_bbb_120',
'r_sma5_bbp_120',
'r_dmp_adx_120',
'r_dmn_adx_120',
'r_rsi_3_5',
'r_rsi_3_10',
'r_rsi_5_10',
'r_rsi_5_15',
'r_rsi_10_15',
'r_rsi_10_20',
'r_rsi_15_20',
'r_rsi_15_60',
'r_rsi_20_60',
'r_rsi_20_120',
'r_rsi_60_120',
'r_ema_close_3',
'r_ema_close_5',
'r_ema_3_5',
'r_ema_3_10',
'r_ema_5_10',
'r_ema_5_15',
'r_ema_10_15',
'r_ema_10_20',
'r_ema_15_20',
'r_ema_15_60',
'r_ema_20_60',
'r_ema_20_120',
'r_ema_60_120',
'r_stoch_k_d',
'up5',
'down5',
'up10',
'down10',
'up20',
'down20',
'up60',
'down60',
'up120',
'down120',
'r_up_down5',
'r_up_down10',
'r_up_down20',
'r_up_down60',
'r_up_down120',
'r_curr_vol_5',
'r_curr_vol_10',
'r_curr_vol_20',
'r_curr_vol_60',
'r_curr_vol_120',
'intraday_direction',
'perc_max_spread_to_close',
'perc_max_spread_to_open',
'r_av_v5_10',
'r_av_v5_20',
'r_av_v5_60',
'r_av_v10_20',
'r_av_v10_60',
'r_av_v10_120',
'r_av_v20_60',
'r_av_v20_120',
'r_av_v60_120',]

In [10]:
col_target = 'inc_close3max_perc5'
col_target_dec = 'dec_close3min_perc5'

In [11]:
col_target_sig = 'inc_dec'
data[col_target_sig] = 0
data.loc[(data[col_target]==1)|(data[col_target_dec]==1), col_target_sig] = 1

# Train test split

In [12]:
count_train = int(data.shape[0]*0.75//1)
count_test = int(data.shape[0] - count_train)

In [13]:
print(f"{count_train} {count_test}")

4536 1512


In [14]:
index_train = data.index[0:count_train]
index_test = data.index[count_train:]

In [15]:
# from sklearn.model_selection import train_test_split

# train_idx, valid_idx, y_train, y_valid = train_test_split(index_train, data.loc[index_train, col_target], test_size=0.25, random_state=42, stratify=data.loc[index_train, col_target])

In [16]:
# len(train_idx)

In [17]:
len(valid_idx)

NameError: name 'valid_idx' is not defined

# Logistic Regression

## Feature Transformation

### Discritization

In [ ]:
from feature_engine.discretisation import DecisionTreeDiscretiser

data_used= data[cols_feat + [col_target]].copy()
data_used[cols_feat] = data[cols_feat].fillna(-999.99)

disc = DecisionTreeDiscretiser(bin_output="prediction",
                               cv=5,
                               scoring='roc_auc',
                               variables=cols_feat,
                               param_grid={'max_leaf_nodes': [2,3,4]},
                               regression=False)

disc.fit(data_used.loc[index_train, cols_feat], data_used.loc[index_train, col_target])

In [ ]:
data_t = disc.transform(data_used[cols_feat])

In [ ]:
del data_used
gc.collect()

### WoE

In [ ]:
from feature_engine.encoding import WoEEncoder, RareLabelEncoder

# set up a weight of evidence encoder
woe_encoder = WoEEncoder(
    variables=cols_feat,
    ignore_format=True,
)

# set up a rare label encoder
rare_encoder = RareLabelEncoder(
    tol=0.05,
    n_categories=4,
    variables=cols_feat,
    ignore_format=True,
)

# fit and transform data
rare_encoder.fit(data_t.loc[index_train, cols_feat])
data_tr = rare_encoder.transform(data_t[cols_feat])

# fit the encoder
woe_encoder.fit(data_tr.loc[index_train, cols_feat], data.loc[index_train, col_target])

In [ ]:
data_tr['RSI_3'].value_counts()
# data_tr.head()

## Feature Selection

#### High univariate AUC + Correlation

In [ ]:
corr_mat_lr = data_t.loc[index_train].corr()

In [ ]:
from sklearn.metrics import roc_auc_score

feature_auc = pd.Series(index=cols_feat)

for icol in cols_feat:
    temp_mask = (data_t.index.isin(index_train)) & (data[col_target].isnull()==False) & (data[icol].isnull()==False)
    feature_auc.loc[icol] = roc_auc_score(data.loc[temp_mask, col_target], data_t.loc[temp_mask, icol])

In [ ]:
feature_auc.loc[feature_auc<0.5] = 1 - feature_auc.loc[feature_auc<0.5]
feature_auc.sort_values(ascending=False, inplace=True)
display(feature_auc)

In [ ]:
use_feats = feature_auc.index.tolist()
def corr_gain(list_feat, corr_mat, list_use, thc=0.4):
    if len(list_feat)>0:
        mat_corr = corr_mat.loc[list_feat[0], list_feat[1:]].abs()
        del_mat = mat_corr.loc[mat_corr>thc].index.to_list()
        list_feat = [ifeat for ifeat in list_feat if ifeat not in del_mat]
        list_use.append(list_feat[0])
        list_feat = list_feat[1:]
        corr_gain(list_feat, corr_mat, list_use)
    else:
        return None

In [ ]:
list_use = list()
list_feat = feature_auc.index.tolist()

corr_gain(list_feat, corr_mat_lr, list_use)
len(list_use)

### LogReg feature selection

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris  # Example dataset; replace with your dataset
from sklearn.metrics import accuracy_score

# Load example dataset
X = pd.DataFrame(data_t, columns=list_use)
y = data[col_target]

# Initialize variables
selected_features = []  # Keep track of selected features
remaining_features = list(X.columns)  # Start with all features as candidates
best_score = 0  # Best score initialized to 0

# Logistic Regression Model
model = LogisticRegression(max_iter=1000)

# Forward Feature Selection
while remaining_features:
    scores = []
    for feature in remaining_features:
        # Select current features (selected + new candidate feature)
        current_features = selected_features + [feature]
        # Perform cross-validation with the current feature set
        cv_score = np.mean(cross_val_score(model, X[current_features], y, cv=5, scoring='roc_auc'))
        scores.append((cv_score, feature))

    # Select the feature with the highest score
    scores.sort(reverse=True)
    best_new_score, best_new_feature = scores[0]

    # Check if adding this feature improves performance
    if best_new_score > best_score:
        best_score = best_new_score
        selected_features.append(best_new_feature)
        remaining_features.remove(best_new_feature)
        print(f"Feature '{best_new_feature}' added with score: {best_new_score}")
    else:
        # Stop if no improvement
        print("No improvement, stopping feature selection.")
        break

print(f"\nSelected features: {selected_features}")
print(f"Best cross-validated score: {best_score}")

In [ ]:
feats_lr = selected_features[:6]

## Modelling

In [ ]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from optuna.samplers import TPESampler

# Load dataset
X = pd.DataFrame(data_t.loc[index_train], columns=list_use)
y = data.loc[index_train, col_target]

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameter search space
    C = trial.suggest_loguniform('C', 1e-5, 1e2)
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    penalty = 'l2'
    
    # Create Logistic Regression model with trial parameters
    model = LogisticRegression(C=C, solver=solver, penalty=penalty, max_iter=1000)
    
    # Use a pipeline to scale features
    pipeline = Pipeline([('scaler', StandardScaler()), ('lr', model)])
    
    # Evaluate model with cross-validation
    score = cross_val_score(pipeline, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Create study and optimize
study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)

In [ ]:
print("Best hyperparameters:", study.best_params)
print("Best cross-validation accuracy:", study.best_value)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
X = pd.DataFrame(data_t.loc[index_train], columns=list_use)
y = data.loc[index_train, col_target]


# Initialize Logistic Regression model
model = LogisticRegression(max_iter=1000, C=study.best_params['C'])

# Train the model
model.fit(X, y)

# Make predictions on the test set
data['prediction'] = model.predict_proba(data_t[list_use])[:,1]

# Evaluate model performance
accuracy = roc_auc_score(data.loc[index_test, col_target], data.loc[index_test, 'prediction'])

print("Accuracy:", accuracy)

In [ ]:
data['cut_prediction'] = np.nan
data.loc[index_test, 'cut_prediction'] = pd.cut(data.loc[index_test, 'prediction'], 10, labels=False)
data[['cut_prediction', 'prediction',  col_target]].groupby('cut_prediction').agg(['count', 'mean'])

# Random Forest

### Feature Selection

In [18]:
corr_mat = data.loc[index_train].corr()

In [19]:
from sklearn.ensemble import RandomForestClassifier

X = pd.DataFrame(data.loc[index_train, cols_feat], columns=cols_feat)
y = data.loc[index_train, col_target]

clf = RandomForestClassifier(random_state=123)
clf.fit(X, y)

feature_importances = pd.Series(clf.feature_importances_, index=cols_feat)
feature_importances = feature_importances.sort_values(ascending=False)

In [20]:
def corr_gain(list_feat, corr_mat, list_use, thc=0.4):
    if len(list_feat)>0:
        mat_corr = corr_mat.loc[list_feat[0], list_feat[1:]].abs()
        del_mat = mat_corr.loc[mat_corr>thc].index.to_list()
        list_feat = [ifeat for ifeat in list_feat if ifeat not in del_mat]
        list_use.append(list_feat[0])
        list_feat = list_feat[1:]
        corr_gain(list_feat, corr_mat, list_use)
    else:
        return None

In [21]:
list_use = list()
list_feat = feature_importances.index.tolist()

corr_gain(list_feat, corr_mat, list_use)
len(list_use)

32

In [22]:
list_use

['perc_max_spread_to_open',
 'OBV',
 'r_sma5_bbb_5',
 'r_av_v5_60',
 'r_dmp_adx_60',
 'r_low_bbb_3',
 'r_dmp_adx_5',
 'r_dmn_adx_15',
 'diff_BBM_20',
 'diff_BBU_60',
 'grad_BBB_10',
 'grad_BBB_20',
 'grad_BBB_120',
 'r_sma5_bbp_120',
 'grad_BBL_5',
 'grad_BBP_3',
 'r_curr_vol_5',
 'r_sma3_bbl_120',
 'grad_BBU_15',
 'intraday_direction',
 'r_sma3_bbl_60',
 'grad_BBU_3',
 'r_open_bbp_60',
 'r_close_bbp_15',
 'r_open_bbp_10',
 'r_low_bbl_15',
 'r_low_bbp_20',
 'r_high_rsi_3',
 'grad_DMN_10',
 'r_up_down120',
 'r_sma3_bbm_3',
 'r_sma5_bbm_5']

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Load example dataset
X = pd.DataFrame(data.loc[index_train, list_use], columns=list_use)
y = data.loc[index_train, col_target]

# Initialize variables
selected_features = []  # List to hold selected features
remaining_features = list(X.columns)  # Start with all features as candidates
best_score = 0  # Track the best score (AUC)

# Random Forest model
model = RandomForestClassifier(random_state=42)

# Iterative Feature Selection based on ROC AUC
while len(remaining_features)>0:
    print(len(remaining_features))
    scores = []
    for feature in remaining_features:
        # Define the feature set to test (selected features + candidate feature)
        current_features = selected_features + [feature]
        # Perform cross-validation to evaluate AUC
        auc_score = np.mean(cross_val_score(model, X[current_features], y, cv=5, scoring='roc_auc'))
        scores.append((auc_score, feature))

    # Select the feature that resulted in the best AUC score
    scores.sort(reverse=True)
    best_new_score, best_new_feature = scores[0]

    # Check if adding this feature improves the score
    if best_new_score > best_score:
        best_score = best_new_score
        selected_features.append(best_new_feature)
        remaining_features.remove(best_new_feature)
        print(f"Feature '{best_new_feature}' added with AUC score: {best_new_score}")
    else:
        # Stop if no improvement
        print("No improvement, stopping feature selection.")
        break

print(f"\nSelected features: {selected_features}")
print(f"Best cross-validated AUC score: {best_score}")

32
Feature 'perc_max_spread_to_open' added with AUC score: 0.6734670567427145
31
Feature 'OBV' added with AUC score: 0.740701466107894
30
Feature 'intraday_direction' added with AUC score: 0.7754403141482659
29
Feature 'diff_BBU_60' added with AUC score: 0.779063749259867
28
Feature 'grad_BBP_3' added with AUC score: 0.7854508100682757
27
Feature 'r_sma5_bbm_5' added with AUC score: 0.7875134002004147
26
No improvement, stopping feature selection.

Selected features: ['perc_max_spread_to_open', 'OBV', 'intraday_direction', 'diff_BBU_60', 'grad_BBP_3', 'r_sma5_bbm_5']
Best cross-validated AUC score: 0.7875134002004147


### Hyperparameter Tuning

In [24]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from optuna.samplers import TPESampler

# Load dataset
X = pd.DataFrame(data.loc[index_train, selected_features], columns=selected_features)
y = data.loc[index_train, col_target]

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameter search space
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    
    # Initialize the Random Forest model with trial parameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42,
    )
    
    # Evaluate model with cross-validation using ROC AUC as the metric
    score = cross_val_score(model, X, y, cv=5, scoring='roc_auc').mean()
    return score

# Create Optuna study and optimize
study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=100)

# Display best hyperparameters
print("Best hyperparameters:", study.best_params)
print("Best cross-validated ROC AUC score:", study.best_value)


[I 2024-10-26 22:42:56,790] A new study created in memory with name: no-name-d0a846e7-12ca-4539-ab02-8d7bc7d92e72
[I 2024-10-26 22:43:20,442] Trial 0 finished with value: 0.7949183778331317 and parameters: {'n_estimators': 282, 'max_depth': 11, 'min_samples_split': 4, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 0 with value: 0.7949183778331317.
[I 2024-10-26 22:43:22,942] Trial 1 finished with value: 0.7798645869942384 and parameters: {'n_estimators': 52, 'max_depth': 18, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7949183778331317.
[I 2024-10-26 22:43:30,161] Trial 2 finished with value: 0.806026110203306 and parameters: {'n_estimators': 210, 'max_depth': 7, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 2 with value: 0.806026110203306.
[I 2024-10-26 22:43:33,733] Trial 3 finished with value: 0.784237300108933 and parameters: {'n_estimators': 195, 'max_depth': 2, 'min_samp

Best hyperparameters: {'n_estimators': 220, 'max_depth': 10, 'min_samples_split': 11, 'min_samples_leaf': 10, 'max_features': 'sqrt'}
Best cross-validated ROC AUC score: 0.8140032104274072


In [25]:
# Display best hyperparameters
print("Best hyperparameters:", study.best_params)
print("Best cross-validated ROC AUC score:", study.best_value)

Best hyperparameters: {'n_estimators': 220, 'max_depth': 10, 'min_samples_split': 11, 'min_samples_leaf': 10, 'max_features': 'sqrt'}
Best cross-validated ROC AUC score: 0.8140032104274072


### Modelling

In [28]:
model = RandomForestClassifier(
        n_estimators=study.best_params['n_estimators'],
        max_depth=study.best_params['max_depth'],
        min_samples_split=study.best_params['min_samples_split'],
        min_samples_leaf=study.best_params['min_samples_leaf'],
        max_features=study.best_params['max_features'],
        random_state=42,
    )

model.fit(data.loc[index_train, selected_features], data.loc[index_train, col_target])
data['prediction_rf'] = model.predict_proba(data[selected_features])[:,1]

In [30]:
# Evaluate model performance
from sklearn.metrics import roc_auc_score

accuracy = roc_auc_score(data.loc[index_test, col_target], data.loc[index_test, 'prediction_rf'])

print("Accuracy:", accuracy)

Accuracy: 0.6690008282508847


In [31]:
data['cut_prediction_rf'] = np.nan
data.loc[index_test, 'cut_prediction_rf'] = pd.cut(data.loc[index_test, 'prediction_rf'], 10, labels=False)
data[['cut_prediction_rf', 'prediction_rf',  col_target]].groupby('cut_prediction_rf').agg(['count', 'mean'])

prediction_rf           inc_close3max_perc5          
                          count      mean               count      mean
cut_prediction_rf                                                      
0.0                         876  0.029756                 876  0.049087
1.0                         397  0.061181                 397  0.073048
2.0                          90  0.101181                  90  0.100000
3.0                          55  0.143914                  55  0.145455
4.0                          56  0.184329                  56  0.250000
5.0                          20  0.216954                  20  0.200000
6.0                          10  0.266742                  10  0.400000
7.0                           4  0.295616                   4  0.250000
8.0                           2  0.341682                   2  0.500000
9.0                           2  0.400520                   2  0.500000

# K-Means

## Feature Selection

In [ ]:
corr_mat = data.loc[index_train].corr()

In [ ]:
from sklearn.metrics import roc_auc_score

feature_auc = pd.Series(index=cols_feat)

for icol in cols_feat:
    temp_mask = (data.index.isin(index_train)) & (data[col_target].isnull()==False) & (data[icol].isnull()==False)
    feature_auc.loc[icol] = roc_auc_score(data.loc[temp_mask, col_target], data.loc[temp_mask, icol])

In [ ]:
feature_auc.loc[feature_auc<0.5] = 1 - feature_auc.loc[feature_auc<0.5]
feature_auc.sort_values(ascending=False, inplace=True)
display(feature_auc)

In [ ]:
use_feats = feature_auc.index.tolist()
def corr_gain(list_feat, corr_mat, list_use, thc=0.3):
    if len(list_feat)>0:
        mat_corr = corr_mat.loc[list_feat[0], list_feat[1:]].abs()
        del_mat = mat_corr.loc[mat_corr>thc].index.to_list()
        list_feat = [ifeat for ifeat in list_feat if ifeat not in del_mat]
        list_use.append(list_feat[0])
        list_feat = list_feat[1:]
        corr_gain(list_feat, corr_mat, list_use)
    else:
        return None

In [ ]:
list_use = list()
list_feat = feature_auc.index.tolist()

corr_gain(list_feat, corr_mat, list_use)
len(list_use)

## k-means

In [ ]:
import matplotlib.cm as cm
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X, y = make_blobs(
    n_samples=500,
    n_features=2,
    centers=4,
    cluster_std=1,
    center_box=(-10.0, 10.0),
    shuffle=True,
    random_state=1,
)  # For reproducibility

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
X = data.loc[:, list_use[:20]].dropna().values

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

In [ ]:
data['cluster'] = np.nan
n_feat=30
n_cluster = 3
kmeans = KMeans(n_clusters=n_cluster, random_state=10, n_init="auto").fit(data.loc[:, list_use[:n_feat]].dropna())
data.loc[data[list_use[:n_feat]].dropna().index, 'cluster'] = kmeans.predict(data.loc[data[list_use[:n_feat]].dropna().index, list_use[:n_feat]])
data['cluster'] = data['cluster'].fillna(-1)

In [ ]:
data.loc[index_test, ['cluster', col_target]].groupby('cluster').agg(['count', 'mean'])

In [ ]:
data[col_target].mean()

## LogReg LEarning

In [ ]:
col_predict = f"proba_{col_target}"

In [ ]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def objective(trial):
    cs_ = trial.suggest_loguniform('C', 1e-10, 1e10)
    clf = LogisticRegression(C=cs_)
    return cross_val_score(clf, data_t.loc[valid_idx, cols_use_feat], data.loc[valid_idx, col_target], n_jobs=-1, cv=5, scoring='roc_auc').mean()

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

In [ ]:
# Print the best set of hyperparameters
print('Best hyperparameters: ', study.best_params)
# Print the corresponding performance
print('Best performance: ', study.best_value)

In [ ]:
clf_logreg = LogisticRegression(C=1*10e1)
clf_logreg.fit(data_t.loc[train_idx, cols_use_feat], data.loc[train_idx, col_target])

In [ ]:
data[col_predict] = clf_logreg.predict_proba(data_t[cols_use_feat])[:,1]

In [ ]:
roc_auc_score(data.loc[index_test, col_target], data.loc[index_test, col_predict])

In [ ]:
roc_auc_score(data.loc[valid_idx, col_target], data.loc[valid_idx, col_predict])

In [ ]:
roc_auc_score(data.loc[train_idx, col_target], data.loc[train_idx, col_predict])

In [ ]:
data.loc[data['day']==20241018]

In [ ]:
data['cut_predict'] = np.nan
data.loc[index_test, 'cut_predict'] = pd.cut(data.loc[index_test, col_predict], 10, labels=False)
data.loc[index_test, ['cut_predict', col_predict, col_target]].groupby('cut_predict').agg(['count','mean'])

In [ ]:
data[col_predict].value_counts()